# Clustering para segmentación de clientes

El objetivo de este análisis de clustering es identificar grupos de clientes con características similares, lo que permitirá a la empresa personalizar las ofertas y mejorar la retención de los clientes, reduciendo las cancelaciones y optimizando las promociones.

In [ ]:
# Importación de paquetes
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../data/halcon_viajes_limpios.csv") # Cargar el dataset

### Paso 1: Preparación de los datos

In [ ]:
# Seleccionar las columnas relevantes para el clustering
variables = ['edad', 'antelacion_reserva', 'duracion_viaje', 'costo_total', 
            'numero_personas', 'tipo_paquete', 'clase_vuelo']

# Filtrar el dataset para que solo contenga las columnas seleccionadas
df_clustering = df[variables]

# Aplicar One-Hot Encoding a las columnas categóricas 'tipo_paquete' y 'clase_vuelo'
df_clustering = pd.get_dummies(df_clustering, columns=['tipo_paquete', 'clase_vuelo'], drop_first=True)

# Escalar las variables numéricas
numerical_features = ['edad', 'antelacion_reserva', 'duracion_viaje', 'costo_total', 'numero_personas']
scaler = StandardScaler()
df_clustering[numerical_features] = scaler.fit_transform(df_clustering[numerical_features])

# Verificar cómo quedaron las columnas
df_clustering.head()

### Paso 2: Aplicación de K-Means para Clustering

In [ ]:
# Aplicar K-Means con 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)

# Ajustar el modelo
df_clustering['cluster'] = kmeans.fit_predict(df_clustering)

# Visualizar los resultados con un gráfico de dispersión de los dos primeros componentes principales
plt.figure(figsize=(8,6))
plt.scatter(df_clustering['edad'], df_clustering['costo_total'], c=df_clustering['cluster'], cmap='viridis')
plt.xlabel('Edad')
plt.ylabel('Costo Total')
plt.title('Segmentación de Clientes por Edad y Costo Total')
plt.colorbar(label='Cluster')
plt.show()

# Ver las primeras filas con la asignación de clusters
df_clustering.head()

### Paso 3: Análisis de la relación entre los clusters y las cancelaciones

In [ ]:
# Asegurarnos de que estamos utilizando la columna 'cancelacion_reserva' desde el dataFrame original
df_clustering['cancelacion_reserva'] = df['cancelacion_reserva']

# Analizar la relación entre los clusters y la tasa de cancelaciones
cluster_cancelations = df_clustering.groupby('cluster')['cancelacion_reserva'].value_counts().unstack()

# Mostrar la distribución de cancelaciones en cada cluster
cluster_cancelations.plot(kind='bar', stacked=True, colormap='coolwarm', figsize=(8,6))
plt.title('Distribución de Cancelaciones por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Cantidad de Clientes')
plt.xticks(rotation=0)
plt.legend(['No Cancelado', 'Cancelado'])
plt.show()

### Paso 4: Distribución de los clusters por tipo de paquete y clase de vuelo

In [ ]:
# Gráfico de distribución de los clusters por tipo de paquete
plt.figure(figsize=(8,6))
df_clustering.groupby('cluster')['tipo_paquete_vuelo_hotel_tour'].value_counts().unstack().plot(kind='bar', stacked=True, colormap='viridis')
plt.title('Distribución de Clusters por Tipo de Paquete')
plt.xlabel('Cluster')
plt.ylabel('Cantidad de Clientes')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Gráfico de distribución de los clusters por clase de vuelo (ejecutiva y primera clase)
plt.figure(figsize=(8,6))
df_clustering.groupby('cluster')['clase_vuelo_ejecutiva'].value_counts().unstack().plot(kind='bar', stacked=True, colormap='viridis')
df_clustering.groupby('cluster')['clase_vuelo_primera_clase'].value_counts().unstack().plot(kind='bar', stacked=True, colormap='plasma', alpha=0.5)
plt.title('Distribución de Clusters por Clase de Vuelo')
plt.xlabel('Cluster')
plt.ylabel('Cantidad de Clientes')
plt.xticks(rotation=0)
plt.legend(['Ejecutiva', 'Primera Clase'])
plt.show()

### Paso 5: Perfiles de los Clusters

In [ ]:
# Ver las características promedio de los clientes en cada cluster
cluster_profiles = df_clustering.groupby('cluster').mean()

# Mostrar las características promedio por cluster
print(cluster_profiles)

## Conclusiones del análisis de clustering para segmentación de clientes

**Cluster 0: Clientes impulsivos y de bajo gasto**
- Perfil: Este grupo está compuesto principalmente por clientes jóvenes, que tienden a hacer reservas de bajo costo y prefieren viajes cortos. La antelación de reserva es baja, lo que sugiere una tendencia impulsiva en sus decisiones.

- Comportamiento de Cancelación: Este cluster presenta una alta tasa de cancelación, lo que indica que los clientes en este grupo pueden ser más propensos a cambiar o cancelar sus planes de viaje.

- Recomendación: Para reducir las cancelaciones en este grupo, se podrían ofrecer promociones atractivas o descuentos especiales para incentivar la fidelidad y el compromiso con las reservas. 

**Cluster 1: Clientes premium y planificadores**
- Perfil: Este grupo está compuesto por clientes de mayor edad, con un poder adquisitivo más alto. Prefieren realizar sus reservas con anticipación y optan por paquetes completos como vuelo + hotel + tour. Además, tienden a elegir clases superiores de vuelo (ejecutiva y primera clase).

- Comportamiento de Cancelación: A pesar de su preferencia por planificar sus viajes con tiempo, este grupo tiene una tasa de cancelación baja, aunque no nula.

- Recomendación: Este grupo podría beneficiarse de una personalización más específica en las ofertas. Se podrían ofrecer paquetes exclusivos o de lujo, destacando beneficios adicionales.

**Cluster 2: Clientes intermedios y comprometidos**
- Perfil: Los clientes en este grupo son de edad media y tienden a realizar reservas de bajo costo. Prefieren viajes más largos y, aunque optan por paquetes más simples, muestran un comportamiento de reserva comprometido.

- Comportamiento de Cancelación: Este grupo tiene una baja tasa de cancelación, lo que indica que los clientes son relativamente comprometidos con sus reservas y tienen menos probabilidades de cancelarlas.

- Recomendación: Para mantener el alto nivel de fidelidad de este grupo, se podrían ofrecer paquetes intermedios o paquetes personalizados. También se podrían ofrecer incentivos para aumentar el gasto, como descuentos en actividades adicionales o servicios.

El análisis de clustering ha permitido **segmentar a los clientes en grupos con características y comportamientos de compra similares**, lo que proporciona insights valiosos para **personalizar ofertas y ajustar estrategias de marketing** basadas en el comportamiento de los clientes. De esta manera, conseguimos mejorar la experiencia del cliente y reducir las cancelaciones.